In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import xlrd
from py2neo import Graph,Node,Relationship
import re

In [2]:
# option_entities_file = 'entities.csv'
# option_relations_file = 'relations.txt'
# option_properties_file = 'properties.txt'
option_deleteAllFlag = 1
option_demoFlag = 1
option_read_csv_row =  -1 # -1 for all rows, positive number k for first k rows.
option_userName = 'neo4j'
option_passWord = '123123'
option_config_file = r''
option_entities_dir = ''
option_column_prefix = "" 
option_pas = False
option_intesrv = True


if option_pas == True:
#     option_config_file = r'E:/天远/知识图谱/天远知识图谱文档/数据血缘/PAS/PAS_config.xlsx'
    option_config_file = r'./config/PAS_config.xlsx'
    option_entities_dir = './data/pas/'
    option_column_prefix = "" 
    
if option_intesrv == True:
    option_config_file = r'E:/天远/知识图谱/天远知识图谱文档/天远百度/config/config_v2.0.xlsx'
#     option_config_file = './config1.3.xlsx'
    option_entities_dir = './data/intesrv_real/'
    option_column_prefix = "plt_" 
a = 1 

##### debug

## 读取配置表

### 主外键对应表

In [3]:
df = pd.read_excel(open(option_config_file, 'rb'),sheet_name='主外键对应表')
for col in df.columns:
    df[col] = df[col].astype(str).str.lower()
df['primary_table'] = df['主键'].apply(lambda x: x.split('.')[0])
df['primary_key'] = option_column_prefix + df['主键'].apply(lambda x: x.split('.')[1])
df['foreign_table'] = df['外键'].apply(lambda x: x.split('.')[0])
df['foreign_key'] = option_column_prefix + df['外键'].apply(lambda x: x.split('.')[1])
df_foreign = df
df_foreign

,主键,外键,primary_table,primary_key,foreign_table,foreign_key
0,plt_intesrv_tywork.oid,plt_intesrv_tyworkdispatch.tyworkoid,plt_intesrv_tywork,plt_oid,plt_intesrv_tyworkdispatch,plt_tyworkoid
1,plt_intesrv_tywork.oid,plt_intesrv_tyjob.tyworkoid,plt_intesrv_tywork,plt_oid,plt_intesrv_tyjob,plt_tyworkoid
2,plt_intesrv_tywork.oid,plt_intesrv_typartdeliveryinfo.tyworkoid,plt_intesrv_tywork,plt_oid,plt_intesrv_typartdeliveryinfo,plt_tyworkoid
3,plt_intesrv_typartdeliveryinfo.oid,plt_intesrv_typartdelivery.typartdeliveryinfooid,plt_intesrv_typartdeliveryinfo,plt_oid,plt_intesrv_typartdelivery,plt_typartdeliveryinfooid
4,plt_intesrv_tycustomer.oid,plt_intesrv_tydevice.tycustomeroid,plt_intesrv_tycustomer,plt_oid,plt_intesrv_tydevice,plt_tycustomeroid
5,plt_intesrv_tycustomer.oid,plt_intesrv_tylinkman.customeroid,plt_intesrv_tycustomer,plt_oid,plt_intesrv_tylinkman,plt_customeroid
6,plt_intesrv_tyworkdispatch.oid,plt_intesrv_tyworkmileage.tyworkdispatchoid,plt_intesrv_tyworkdispatch,plt_oid,plt_intesrv_tyworkmileage,plt_tyworkdispatchoid
7,plt_intesrv_tydevice.oid,plt_intesrv_tywork.tydeviceoid,plt_intesrv_tydevice,plt_oid,plt_intesrv_tywork,plt_tydeviceoid
8,plt_org_user.oid,plt_intesrv_typartdeliveryinfo.stockupoid,plt_org_user,plt_oid,plt_intesrv_typartdeliveryinfo,plt_stockupoid
9,plt_org_user.oid,plt_intesrv_tywork.creator,plt_org_user,plt_oid,plt_intesrv_tywork,plt_creator


#### 根据主外键关系，对表进行拓扑排序

In [4]:
tables_name = []
df_foreign_tobedeleted = df_foreign.copy()
count = 0
while len(df_foreign_tobedeleted) > 0:
#     print(count)
    a = df_foreign_tobedeleted['primary_table'].unique()
    b = df_foreign_tobedeleted['foreign_table'].unique()
    c = np.concatenate((a,b))
    tables_name_unordered = set(c)
#     print(tables_name_unordered)
    foreign_table = set(df_foreign_tobedeleted['foreign_table'])
    table = tables_name_unordered.difference(foreign_table).pop()
#     print(tables_name_unordered.difference(foreign_table))
    tables_name.append(table)
#     print(table)
    index_tobedeleted = df_foreign_tobedeleted.loc[df_foreign_tobedeleted['primary_table'] == table].index
#     print(index_tobedeleted)
    df_foreign_tobedeleted.drop(index_tobedeleted, inplace=True)
#     print(df_foreign_tobedeleted[['primary_table','foreign_table']])
    count += 1    

df_foreign_tobedeleted = df_foreign.copy()
a = df_foreign_tobedeleted['primary_table'].unique()
b = df_foreign_tobedeleted['foreign_table'].unique()
c = np.concatenate((a,b))
tables_name_unordered = set(c)
table_not_added = tables_name_unordered.difference(set(tables_name))
while len(table_not_added) > 0:
    tables_name.append(table_not_added.pop())
tables_name

['plt_intesrv_tycustomer',
 'plt_org_user',
 'plt_intesrv_tysrvperson',
 'plt_intesrv_tydevice',
 'plt_intesrv_tywork',
 'plt_intesrv_typartdeliveryinfo',
 'plt_intesrv_tyworkdispatch',
 'plt_intesrv_tylinkman',
 'plt_intesrv_tyjob',
 'plt_intesrv_typartdelivery',
 'plt_intesrv_tyworkmileage']

##### debug

### 关系表

In [5]:
df = pd.read_excel(open(option_config_file, 'rb'),sheet_name='关系表')
for col in df.columns:
    df[col] = df[col].astype(str).str.lower()
df = df.rename(index=str, columns={"关系主语":"sub",
                                   "关系谓语":"predicate",
                                   "关系宾语":"obj",
                                   "关系类型":"type",
                                   "关系所在数据库表":"table"})
df_relation = df
df_relation

,id,sub,predicate,obj,table,type
0,1,联系人,属于,人,nan,nan
1,2,联系人,具有,联系优先级,nan,nan
2,3,客户,具有,联系人,plt_intesrv_tylinkman,nan
3,4,客户,使用,发票模板,nan,nan
4,5,客户,具有,信誉信息,nan,nan
5,6,客户,具有,客户满意度,nan,nan
6,7,挖掘机,属于,吨级,plt_intesrv_tydevice,nan
7,8,挖掘机,属于,客户,plt_intesrv_tydevice,nan
8,9,挖掘机,属于,车龄,nan,nan
9,10,挖掘机,属于,质保状态,plt_intesrv_tydevice,nan


### 概念表

In [6]:
df = pd.read_excel(open(option_config_file, 'rb'),sheet_name='模式图的概念与数据库对应表')
for col in df.columns:
    df[col] = df[col].astype(str).str.lower()
    df[col] = df[col].str.replace("\n","")
df = df.rename(index=str, columns={"模式图中概念":"concept_ch",
                                   "显示名":"display_name",
                                   "数据库表/属性":"table_and_column",
                                   "类型":"type",
                                   "逻辑代码连接":"code",
                                   "内部属性":"inner_properties_ch",
                                   "内部属性对应数据库属性":"properties_col_str"})
# print(df)
df['table'] = None
df['col'] = None
# df[df['type'] == '直接']['table'] = df[df['type'] == '直接']['table_and_column'].astype(str).apply(lambda x: x.split('.')[0])
df.loc[df['type'] == '直接','table'] = df[df['type'] == '直接']['table_and_column'].astype(str).apply(lambda x: x.split('.')[0])
df.loc[df['type'] == '直接','col'] = option_column_prefix + df[df['type'] == '直接']['table_and_column'].astype(str).apply(lambda x: x.split('.')[1])
# df.loc[df['type'] == '直接','properties_col'] = df[df['type'] == '直接']['properties_col_str'].astype(str).apply(lambda x: [option_column_prefix+y for y in x.split(",，") if x != "nan"] )
df.loc[df['type'] == '直接','properties_col'] = df[df['type'] == '直接']['properties_col_str'].astype(str).apply(lambda x: [option_column_prefix+y for y in re.split(",|，", x) if x != "nan"] )
# df.loc[df['type'] == '直接','properties_col_ch'] = df[df['type'] == '直接']['inner_properties_ch'].astype(str).apply(lambda x: [y for y in x.split(",，") if x != "nan"] )
df.loc[df['type'] == '直接','properties_col_ch'] = df[df['type'] == '直接']['inner_properties_ch'].astype(str).apply(lambda x: [y for y in re.split(",|，", x) if x != "nan"] )
df_concept = df
df_concept

,concept_ch,display_name,table_and_column,type,inner_properties_ch,properties_col_str,code,table,col,properties_col,properties_col_ch
0,客户,客户名称,plt_intesrv_tycustomer.oid,直接,"客户号,客户名称,工地位置,办公地位置,住所位置,客户类型，客户等级","id,customername，mainlonlat,customeraddress,hom...",nan,plt_intesrv_tycustomer,plt_oid,"[plt_id, plt_customername, plt_mainlonlat, plt...","[客户号, 客户名称, 工地位置, 办公地位置, 住所位置, 客户类型, 客户等级]"
1,联系人,联系人姓名,plt_intesrv_tylinkman.oid,直接,"联系人号，联系人姓名,联系人电话，角色","id,linkmanname,linkmanphone,linkmantype",nan,plt_intesrv_tylinkman,plt_oid,"[plt_id, plt_linkmanname, plt_linkmanphone, pl...","[联系人号, 联系人姓名, 联系人电话, 角色]"
2,工单,工单号,plt_intesrv_tywork.oid,直接,"工单号,名称，是否停机,工单注册时间，挖机的工作小时数，故障发生时间，工作开始时间，工作结束时间","id,workname，isshutdown,createtime,workhours，fa...",nan,plt_intesrv_tywork,plt_oid,"[plt_id, plt_workname, plt_isshutdown, plt_cre...","[工单号, 名称, 是否停机, 工单注册时间, 挖机的工作小时数, 故障发生时间, 工作开始..."
3,当前八级报警级别,级别,nan,合成,级别,nan,nan,None,None,NaN,NaN
4,历史八级报警级别,nan,nan,合成,级别,nan,nan,None,None,NaN,NaN
5,当前工单状态,unique_id,plt_intesrv_tywork.workstatus,直接,nan,nan,nan,plt_intesrv_tywork,plt_workstatus,[],[]
6,历史工单状态,nan,nan,合成,状态,nan,nan,None,None,NaN,NaN
7,故障解决的去现场次数,nan,nan,合成,次数,nan,nan,None,None,NaN,NaN
8,作业形态,unique_id,plt_intesrv_tywork.workpattern,直接,nan,nan,nan,plt_intesrv_tywork,plt_workpattern,[],[]
9,方案,作业名称,plt_intesrv_tyjob.oid,直接,"作业卡编号,作业名称,作业详情","id,jobname,jobdetails",nan,plt_intesrv_tyjob,plt_oid,"[plt_id, plt_jobname, plt_jobdetails]","[作业卡编号, 作业名称, 作业详情]"


#### 根据主外键关系，对表进行拓扑排序

## 读取实体数据

In [7]:
tables = {}
total_length = 0
for table_name in tables_name:
    print("start to access table " + table_name)
    path = option_entities_dir + table_name + ".csv"
    df = pd.read_csv(path,sep=';',encoding='utf-16')
    for key in list(df.columns.values):  # 如果不这样的话，可能会读入int64的数字，但是neo4j不支持int64
        df[key] = df[key].astype("object")
    print(df.shape) 
    total_length += df.shape[0]
    tables[table_name] = df[:option_read_csv_row]
    print("end to access table " + table_name)
# tables
total_length # 60460

# df = pd.read_csv(option_entities_file,sep=';',encoding='utf-16')
# # print(df.shape)
# # print(df.columns.values)
# for key in list(df.columns.values):  # 如果不这样的话，可能会读入int64的数字，但是neo4j不支持int64
#     df[key] = df[key].astype("object")
# print(df.shape)
# df.head()

start to access table plt_intesrv_tycustomer
(9035, 34)
end to access table plt_intesrv_tycustomer
start to access table plt_org_user
(43, 14)
end to access table plt_org_user
start to access table plt_intesrv_tysrvperson
(15, 17)
end to access table plt_intesrv_tysrvperson
start to access table plt_intesrv_tydevice
(13098, 53)
end to access table plt_intesrv_tydevice
start to access table plt_intesrv_tywork
(444, 86)
end to access table plt_intesrv_tywork
start to access table plt_intesrv_typartdeliveryinfo
(340, 46)
end to access table plt_intesrv_typartdeliveryinfo
start to access table plt_intesrv_tyworkdispatch
(271, 27)
end to access table plt_intesrv_tyworkdispatch
start to access table plt_intesrv_tylinkman
(9053, 13)
end to access table plt_intesrv_tylinkman
start to access table plt_intesrv_tyjob
(7155, 26)
end to access table plt_intesrv_tyjob
start to access table plt_intesrv_typartdelivery
(4029, 41)
end to access table plt_intesrv_typartdelivery
start to access table plt_

43765

## 导入Neo4j

In [8]:
graph = Graph('http://localhost:7474',username=option_userName,password=option_passWord)

In [9]:
if option_deleteAllFlag == 1:
    graph.delete_all()

### 模式图

In [10]:
df_concept.head()

,concept_ch,display_name,table_and_column,type,inner_properties_ch,properties_col_str,code,table,col,properties_col,properties_col_ch
0,客户,客户名称,plt_intesrv_tycustomer.oid,直接,"客户号,客户名称,工地位置,办公地位置,住所位置,客户类型，客户等级","id,customername，mainlonlat,customeraddress,hom...",nan,plt_intesrv_tycustomer,plt_oid,"[plt_id, plt_customername, plt_mainlonlat, plt...","[客户号, 客户名称, 工地位置, 办公地位置, 住所位置, 客户类型, 客户等级]"
1,联系人,联系人姓名,plt_intesrv_tylinkman.oid,直接,"联系人号，联系人姓名,联系人电话，角色","id,linkmanname,linkmanphone,linkmantype",nan,plt_intesrv_tylinkman,plt_oid,"[plt_id, plt_linkmanname, plt_linkmanphone, pl...","[联系人号, 联系人姓名, 联系人电话, 角色]"
2,工单,工单号,plt_intesrv_tywork.oid,直接,"工单号,名称，是否停机,工单注册时间，挖机的工作小时数，故障发生时间，工作开始时间，工作结束时间","id,workname，isshutdown,createtime,workhours，fa...",nan,plt_intesrv_tywork,plt_oid,"[plt_id, plt_workname, plt_isshutdown, plt_cre...","[工单号, 名称, 是否停机, 工单注册时间, 挖机的工作小时数, 故障发生时间, 工作开始..."
3,当前八级报警级别,级别,nan,合成,级别,nan,nan,None,None,NaN,NaN
4,历史八级报警级别,nan,nan,合成,级别,nan,nan,None,None,NaN,NaN


In [11]:
nodes_concept = {}
for index in tqdm(list(df_concept.index)):
    index = int(index)
#     print(index)
#     print(df_concept.iloc[index]['concept_ch'])
    key = df_concept.iloc[index]['concept_ch']
    nodes_concept[key] = Node(key)
    nodes_concept[key].add_label("概念节点")
    for col in list(df_concept.columns.values):
        nodes_concept[key][col] = df_concept[col][index] if df_concept[col][index] == df_concept[col][index] else ""
        
#     nodes_concept[key]['table'] = df_concept.iloc[index]['table_and_column']
    graph.create(nodes_concept[key])

100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [00:01<00:00, 28.81it/s]


In [12]:
df_relation.head()

,id,sub,predicate,obj,table,type
0,1,联系人,属于,人,nan,nan
1,2,联系人,具有,联系优先级,nan,nan
2,3,客户,具有,联系人,plt_intesrv_tylinkman,nan
3,4,客户,使用,发票模板,nan,nan
4,5,客户,具有,信誉信息,nan,nan


In [13]:
for index in tqdm(list(df_relation.index)):
    index = int(index)
#     print(index)
    sub = df_relation.iloc[index]['sub']
    obj = df_relation.iloc[index]['obj']
#     try:
#     print(sub)
#     print(obj)
    aRelation = Relationship(nodes_concept[sub], df_relation.iloc[index]['predicate'], nodes_concept[obj] )
    aRelation["id_relation"] = df_relation.iloc[index]['id']
    graph.create(aRelation)
#     except(KeyError):
#         pass

100%|██████████████████████████████████████████████████████████████████████████████████| 58/58 [00:02<00:00, 28.10it/s]


### 数据图

#### 单个表数据(debug)

#### 所有数据

In [14]:
# if option_deleteAllFlag == 1:
#     graph.delete_all()

# table_name = 'plt_intesrv_tydevice'
for table_name in tables_name:
    print(table_name)
    # 取出属于本表的几个小表，包括实体数据表、概念表、关系表
    df_table = tables[table_name]
    df_concept_one_table = df_concept[df_concept['table'] == table_name]
    df_relation_one_table = df_relation[df_relation['table'] == table_name]

    concept_ch_list_one_table = list(df_concept_one_table['concept_ch'])
    # 对于数据表的每行进行处理
    for i_row in tqdm(range(0,df_table.shape[0])): 
#         print(i_row)
        entity_node = {}
        # 对于概念表中的每个概念进行处理
        for i_entity in list(df_concept_one_table.index):
            value = df_table[ df_concept_one_table['col'][i_entity] ][i_row] # eg:"张三","PC200"
            if value == value: # 如果该节点的值非空
                key = df_concept_one_table['concept_ch'][i_entity] # eg:"客户","挖掘机"
#                 print(key)
                nodeBefore = graph.nodes.match(key, value=value).first()
                if nodeBefore is None:
                    aNode = Node(key)
                    aNode['value'] = value
                    aNode['type'] = key
                    aNode.add_label("实例节点") # Neo4j的官方label，支持高效索引

                    # 为该节点增加对应属性
                    assert len(df_concept_one_table['properties_col'][i_entity]) == len(df_concept_one_table['properties_col_ch'][i_entity])
                    prop_num = len(df_concept_one_table['properties_col'][i_entity])
                    for i_prop in range(0,prop_num):
                        prop_key = df_concept_one_table['properties_col'][i_entity][i_prop]  # eg: prop_key = 'plt_devicebrand'
                        prop_key_ch = df_concept_one_table['properties_col_ch'][i_entity][i_prop] # eg: prop_key_ch = '品牌'
                        prop_value = df_table[prop_key][i_row]
                        if prop_value == prop_value: 
                            aNode[prop_key_ch] = prop_value
                        else: # 属性值如果为空的话，会置为空字符串
                            aNode[prop_key_ch] = ''

                    # 为该节点增加display_name属性
#                     display = "(" + str(key) + ")"
                    display = str(key) + ":"
                    display_col = df_concept_one_table['display_name'][i_entity]
                    if display_col == 'unique_id':
                        display += value 
                    elif display_col == "nan": # 如果那一列为空的话，display就只显示概念名，不显示具体指
                        display_col = display_col[:-2]
                    else:
                        # 根据中文列名找到数据库列名
                        prop_num = len(df_concept_one_table['properties_col'][i_entity])
                        for i_prop in range(0,prop_num):
                            prop_key = df_concept_one_table['properties_col'][i_entity][i_prop]  # eg: prop_key = 'plt_devicebrand'
                            prop_key_ch = df_concept_one_table['properties_col_ch'][i_entity][i_prop] # eg: prop_key_ch = '品牌'
                            if prop_key_ch == display_col:
                                break
                        assert i_prop != prop_num - 1 or (i_prop == prop_num - 1 and prop_key_ch == display_col)
                        display += str(df_table[prop_key][i_row])
#                     print("debug:display=", display)
                    aNode['display_name'] = display

#                     print(aNode)
                    graph.create(aNode)
                else:
                    aNode = nodeBefore
                entity_node[key] = aNode
    #     print(entity_node)
        # 建立本行数据中的节点间的关系
        for i_relation in list(df_relation_one_table.index):
    #         try: 
            # 建立sub主语实体，有可能已经在entity_node中，也有可能是外键
            entity_ch = df_relation_one_table['sub'][i_relation]
            if entity_ch in entity_node.keys(): # 本行数据确实有该节点
                entity = entity_node[entity_ch]
            elif entity_ch not in concept_ch_list_one_table: # 本行数据无该节点，则为外键
                # entity_ch(eg:客户)去模式表里找该概念的数据库主键来源，根据主键和当前表table_name来找到该概念在当前表的外键列名，然后根据该外键列名找到外键具体值，去neo4j里查询该节点，
#                 print(entity_ch)
                primary_key_table = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'table'].item()
                primary_key_col = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'col'].item()
#                 print(primary_key_table)
#                 print(primary_key_col)
#                 print(entity_ch)
#                 print(df_foreign['primary_table'] == primary_key_table)
#                 print((df_foreign['primary_key'] == primary_key_col))
#                 print((df_foreign['foreign_table'] == table_name))
                foreign_key = df_foreign.loc[(df_foreign['primary_table'] == primary_key_table)
                                  & (df_foreign['primary_key'] == primary_key_col)
                                  & (df_foreign['foreign_table'] == table_name), 'foreign_key'].item()
#                 print(foreign_key)
                key = entity_ch
                value = df_table[foreign_key][i_row]
                if value != value: # 虽然该键是外键，但是该键依然可能是控制(玄学数据)
                    continue
                nodeBefore = graph.nodes.match(key, value=value).first()
    #             assert nodeBefore is not None
                if nodeBefore is None: # 这是测试数据的锅，否则的话永远不应该为None的
                    continue
                entity = nodeBefore
    #             print(type(primary_key_table)) 
    #             primary_key_table, primary_key_col
    #             continue
            else: # 本行数据的该节点value为空，所以没有该节点
                continue
            sub = entity

            # 建立obj宾语实体，这二三十航代码都是抄的上面的，只要把首末行的sub改成obj就可以了
            entity_ch = df_relation_one_table['obj'][i_relation]
            if entity_ch in entity_node.keys(): # 本行数据确实有该节点
                entity = entity_node[entity_ch]
            elif entity_ch not in concept_ch_list_one_table: # 本行数据无该节点，则为外键
                # 根据entity_ch(eg:客户)去模式表里找该概念的数据库主键来源，根据主键和当前表table_name来找到该概念在当前表的外键列名，然后根据该外键列名找到外键具体值，去neo4j里查询该节点，
#                 print(entity_ch)
                primary_key_table = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'table'].item()
                primary_key_col = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'col'].item()
#                 print(primary_key_table)
#                 print(primary_key_col)
#                 print(entity_ch)
#                 print(df_foreign['primary_table'] == primary_key_table)
#                 print((df_foreign['primary_key'] == primary_key_col))
#                 print((df_foreign['foreign_table'] == table_name))
#                 print("-"*10)
#                 print(df_foreign.loc[(df_foreign['primary_table'] == primary_key_table)
#                                   & (df_foreign['primary_key'] == primary_key_col)
#                                   & (df_foreign['foreign_table'] == table_name), 'foreign_key'])
#                 print("-"*10)
                foreign_key = df_foreign.loc[(df_foreign['primary_table'] == primary_key_table)
                                  & (df_foreign['primary_key'] == primary_key_col)
                                  & (df_foreign['foreign_table'] == table_name), 'foreign_key'].item()
#                 print(foreign_key)
                key = entity_ch
                value = df_table[foreign_key][i_row]
                if value != value: # 虽然该键是外键，但是该键依然可能是空值(玄学数据)
                    continue
                nodeBefore = graph.nodes.match(key, value=value).first()
    #             assert nodeBefore is not None
                if nodeBefore is None: # 测试数据的锅，否则的话永远不应该为None的
                    continue            
                entity = nodeBefore
    #             print(type(primary_key_table))
    #             primary_key_table, primary_key_col
    #             continue
            else: # 本行数据的该节点value为空，所以没有该节点
                continue
            obj = entity

            # 建立主宾实体间的关系
            predicate = df_relation_one_table['predicate'][i_relation]
            aRelation = Relationship(sub, predicate, obj)
            aRelation['id_relation'] = df_relation_one_table['id'][i_relation]
            graph.create(aRelation)
    #         except KeyError: # 因为某些节点可能因为值为空所以没有该节点，所以在entity_node字典中没有该节点
    #             pass
    #         print(entity_name)

plt_intesrv_tycustomer


100%|██████████████████████████████████████████████████████████████████████████████| 9034/9034 [08:14<00:00, 18.27it/s]


plt_org_user


100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:15<00:00,  5.37it/s]


plt_intesrv_tysrvperson


...

100%|██████████████████████████████████████████████████████████████████████████████| 7154/7154 [08:37<00:00, 10.03it/s]


plt_intesrv_typartdelivery


100%|██████████████████████████████████████████████████████████████████████████████| 4028/4028 [00:52<00:00, 26.73it/s]


plt_intesrv_tyworkmileage


100%|████████████████████████████████████████████████████████████████████████████████| 281/281 [00:11<00:00, 23.59it/s]
